In [1]:
    import abc, jsonschema, rdflib, collections, itertools, operator, copy, functools, pyld.jsonld as jsonld, ast, pathlib, typing
    import strict_rfc3339, rfc3986, rfc3987, IPython, toolz, builtins

In [2]:
    class Schema:
        @classmethod
        def schema(cls): return {**collections.ChainMap(*(getattr(object, '__annotations__', {}) or {} for object in cls.__mro__)), 'title': cls.__name__, }
        @classmethod
        def validate(cls, object, schema=None): jsonschema.validate(object, schema or cls.schema(), format_checker=jsonschema.FormatChecker())

In [3]:
    class JsonSchemaMeta(Schema, abc.ABCMeta):
        def __init_subclass__(cls, **kwargs):  cls.__annotations__ = cls.validate(kwargs) or kwargs

    class JsonSchema(JsonSchemaMeta, **jsonschema.Draft7Validator.META_SCHEMA): ...

In [4]:
    def map_container(callable):
        @functools.wraps(callable)
        def logic(object, *args, **kwargs):
            if isinstance(object, (list, dict)): return object.map(callable, *args, **kwargs)
            return object.pipe(callable, *args, **kwargs)
        return logic

In [12]:
class Parser:
    def text(self): return self
    def read(self): return String(pathlib.Path(self).read_text())
    def yaml(self):
        try: from ruamel import yaml
        except: import yaml
        return self.pipe(operator.methodcaller('text'), yaml.safe_load, Type.object)
    def json(self):
        try: import ujson as json
        except: import json
        return self.pipe(operator.methodcaller('text'), json.loads, Type.object)
    def __import__(self): return __import__('importlib').import_module(self)
    def git(self): return __import__('git').Repo(self)
    def download_json(self, *args, **object): return Type.object(__import__('requests').get(self, *args, **object).json())
    def download(self, *args, **object): return String.object(__import__('requests').get(self, *args, **object).text)
    def graphviz(self): return __import__('graphviz').Source(object)

for self in vars(Parser):
    if self not in dir(object) and (
        self in '__import__'.split() or not self.startswith('_')
    ): setattr(Parser, self, map_container(getattr(Parser, self)))

In [13]:
    class Type(Schema, metaclass=JsonSchema): 
        __annotations__ = {}
        def __new__(cls, object=None, *args, **kwargs):
            if object is None: object = copy.copy(cls.schema().get('default'))
            return cls.validate(object) or super().__new__(cls, object, *args, **kwargs)

        def __init_subclass__(cls, **kwargs):  cls.__annotations__ = type(cls).validate(kwargs) or kwargs

        @classmethod
        def object(type, object=None):
            for cls in (type.__subclasses__()):
                try: return cls.object(object)
                except BaseException as e: ...
            else: return type(object)
            
        def pipe(self, *funcs, **kwargs): return toolz.compose(*reversed(funcs))(self, **kwargs)
        def do(self, *funcs, **kwargs): self.pipe(*funcs, **kwargs); return self
        def list(self): return List(self)
        def tuple(self): return tuple(self)
        def set(self): return set(self)
        def str(self): return String.object(self)
        def zip(self, *args): return list(zip(self, *args))
        def starmap(self, callable): return List(itertools.starmap(callable, self))
        
    class Container(Parser):
        def series(self, *args, **kwargs) -> 'pandas.Series': return __import__('pandas').Series(self, *args, **kwargs)
        def frame(self, *args, **kwargs) -> 'pandas.DataFrame': return __import__('pandas').DataFrame(self, *args, **kwargs)
        def __getitem__(self, object): return Type.object(super().__getitem__(object))
        
    class Sequence(Container):
        def map(self, callable, **kwargs): return self.pipe(toolz.partial(toolz.map, toolz.partial(callable, **kwargs)), list, List)
        def filter(self, callable, **kwargs): return self.pipe(toolz.partial(toolz.filter, toolz.partial(callable, **kwargs)), list, List)
        def reduce(self, callable, **kwargs): return self.pipe(toolz.partial(toolz.reduce, toolz.partial(callable, **kwargs)), Type.object)
        def groupby(self, callable, **kwargs): return self.pipe(toolz.partial(toolz.groupby, toolz.partial(callable, **kwargs)), Dict)
        def reduceby(self, callable, **kwargs): return self.pipe(toolz.partial(toolz.reduceby, toolz.partial(callable, **kwargs)), Dict)

In [14]:
    class Dict(Type, Container, dict, type='object', default={}): 
        def map(self, value=None, key=None): return Dict({(key or toolz.identity)(k): (value or toolz.identity)(v) for k, v in self.items()})
        def filter(self, value=None, key=None): return Dict({k: v for k, v in self.items() if (
            (key(k) if key else True) and (value(v) if value else True)
        )})
        

In [15]:
    class List(Type, Sequence, list, type='array', default=[]): 
        def __new__(cls, object=None, *args, **kwargs): 
            if isinstance(object, tuple): object = list(object)
            return super().__new__(cls, object, *args, **kwargs)        

In [9]:
    class Null(Type, type='null'): 
        def __new__(cls, object=None, *args, **kwargs): return cls.validate(cls, object)
    class Integer(Type, int, type='integer', default=''): ...
    class Number(Type, float, type='number', default=''): ...
    
    class String(Type, Parser, str, type='string', default=''):
        def path(self) -> pathlib.Path: return pathlib.Path(self)
    
    class Date(String, format='date'): ...
    class Datetime(String, format='date-time'): ...
    class Time(String, format='time'): ...
    class Color(String, format='color'): ...
    
    class Email(String, format='email'): ...
    class Uri(String, format='uri'):         
        def text(self, *args, **object): return __import__('requests').get(self,*args, **object).text
        
    class File(String): 
        def text(self): return self.path().read_text()
        @classmethod
        def validate(cls, object, schema=None):
            if object.path().exists(): return 
            raise ValueError(F"{object} is not a file.")
    class Dir(File): 
        @classmethod
        def validate(cls, object, schema=None):
            if object.path().is_dir(): return 
            raise ValueError(F"{object} is not a file.")

In [10]:
    class StringTypes(ast.NodeTransformer):
        def visit_type(self, node, type=None):
            next = ast.parse(F"""__import__('importlib').import_module('{__name__}').{type}.object('')""").body[0].value
            next.args = [node]
            return ast.copy_location(next, node)
        
        visit_JoinedStr = visit_Str = functools.partialmethod(visit_type, type='String')
        visit_DictComp = visit_Dict = functools.partialmethod(visit_type, type='Dict')
        visit_ListComp = visit_List = functools.partialmethod(visit_type, type='List')

In [11]:
    def unload_ipython_extension(shell): shell.ast_transformers = [x for x in shell.ast_transformers if not isinstance(x, StringTypes)]
    def load_ipython_extension(shell): unload_ipython_extension(shell) or shell.ast_transformers.append(StringTypes())
    __name__ == '__main__' and load_ipython_extension(get_ipython())

    __name__ == '__main__' and "jschema.ipynb".yaml()['cells'].series().apply("pandas".imp().Series).T

In [13]:
if __name__ == '__main__':
    __import__('requests_cache').install_cache('jschema')
    "https://api.github.com/users/tonyfast/gists".download_json().frame().T.pipe(display)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
url,https://api.github.com/gists/ab4f3cf987a87d574...,https://api.github.com/gists/79779f89e749c72e7...,https://api.github.com/gists/7b66a5d9b09c4754c...,https://api.github.com/gists/306be744a71ef148d...,https://api.github.com/gists/15c6ac8f005a55222...,https://api.github.com/gists/4f56c72c99fdd3464...,https://api.github.com/gists/e387e567cebf4ec72...,https://api.github.com/gists/d845d7a63ba24aab8...,https://api.github.com/gists/87562b3e76e20855a...,https://api.github.com/gists/6382152537773be2f...,...,https://api.github.com/gists/5a0507d00a1081fad...,https://api.github.com/gists/605e63deb188f6901...,https://api.github.com/gists/61a17214486f1e194...,https://api.github.com/gists/030c51e35e2f5e469...,https://api.github.com/gists/8c278dc253179e0eb...,https://api.github.com/gists/0bf98418128c7823f...,https://api.github.com/gists/1447eb0077eb44e34...,https://api.github.com/gists/98530c6644485022f...,https://api.github.com/gists/0a06a249c8de5d236...,https://api.github.com/gists/8ee0038edad9d8ab4...
forks_url,https://api.github.com/gists/ab4f3cf987a87d574...,https://api.github.com/gists/79779f89e749c72e7...,https://api.github.com/gists/7b66a5d9b09c4754c...,https://api.github.com/gists/306be744a71ef148d...,https://api.github.com/gists/15c6ac8f005a55222...,https://api.github.com/gists/4f56c72c99fdd3464...,https://api.github.com/gists/e387e567cebf4ec72...,https://api.github.com/gists/d845d7a63ba24aab8...,https://api.github.com/gists/87562b3e76e20855a...,https://api.github.com/gists/6382152537773be2f...,...,https://api.github.com/gists/5a0507d00a1081fad...,https://api.github.com/gists/605e63deb188f6901...,https://api.github.com/gists/61a17214486f1e194...,https://api.github.com/gists/030c51e35e2f5e469...,https://api.github.com/gists/8c278dc253179e0eb...,https://api.github.com/gists/0bf98418128c7823f...,https://api.github.com/gists/1447eb0077eb44e34...,https://api.github.com/gists/98530c6644485022f...,https://api.github.com/gists/0a06a249c8de5d236...,https://api.github.com/gists/8ee0038edad9d8ab4...
commits_url,https://api.github.com/gists/ab4f3cf987a87d574...,https://api.github.com/gists/79779f89e749c72e7...,https://api.github.com/gists/7b66a5d9b09c4754c...,https://api.github.com/gists/306be744a71ef148d...,https://api.github.com/gists/15c6ac8f005a55222...,https://api.github.com/gists/4f56c72c99fdd3464...,https://api.github.com/gists/e387e567cebf4ec72...,https://api.github.com/gists/d845d7a63ba24aab8...,https://api.github.com/gists/87562b3e76e20855a...,https://api.github.com/gists/6382152537773be2f...,...,https://api.github.com/gists/5a0507d00a1081fad...,https://api.github.com/gists/605e63deb188f6901...,https://api.github.com/gists/61a17214486f1e194...,https://api.github.com/gists/030c51e35e2f5e469...,https://api.github.com/gists/8c278dc253179e0eb...,https://api.github.com/gists/0bf98418128c7823f...,https://api.github.com/gists/1447eb0077eb44e34...,https://api.github.com/gists/98530c6644485022f...,https://api.github.com/gists/0a06a249c8de5d236...,https://api.github.com/gists/8ee0038edad9d8ab4...
id,ab4f3cf987a87d574144051e652f879a,79779f89e749c72e72b6b4470b512496,7b66a5d9b09c4754c4a7577242975cb1,306be744a71ef148d3590118495ff644,15c6ac8f005a5522291f08d95619e085,4f56c72c99fdd346481a01ad27a9aaf4,e387e567cebf4ec726be9f1bde6503f5,d845d7a63ba24aab895a1580b8702aff,87562b3e76e20855aa076bf793c8208f,6382152537773be2ffc208c3d4e526c2,...,5a0507d00a1081fadccb158d31137d2c,605e63deb188f6901a465420e9e6ce85,61a17214486f1e1947707e1c65ace378,030c51e35e2f5e4698d476443e98765c,8c278dc253179e0ebd1b9ef96b63f51c,0bf98418128c7823f8cf2ecf30bea8fa,1447eb0077eb44e3452a4e3e7d311f70,98530c6644485022f49545f333760916,0a06a249c8de5d236d861237eda846bd,8ee0038edad9d8ab438202d477f26925
node_id,MDQ6R2lzdGFiNGYzY2Y5ODdhODdkNTc0MTQ0MDUxZTY1Mm...,MDQ6R2lzdDc5Nzc5Zjg5ZTc0OWM3MmU3MmI2YjQ0NzBiNT...,MDQ6R2lzdDdiNjZhNWQ5YjA5YzQ3NTRjNGE3NTc3MjQyOT...,MDQ6R2lzdDMwNmJlNzQ0YTcxZWYxNDhkMzU5MDExODQ5NW...,MDQ6R2lzdDE1YzZhYzhmMDA1YTU1MjIy

In [14]:
[*range(10)].filter(lambda x: x%2).map(chr)

ValidationError: <filter object at 0x108538c50> is not of type 'array'

Failed validating 'type' in schema:
    {'default': [], 'title': 'List', 'type': 'array'}

On instance:
    <filter object at 0x108538c50>